In [187]:
#!pip install feedparser
import feedparser
import pandas as pd


In [155]:
#NewsFeed = feedparser.parse("https://www.nhc.noaa.gov/index-at.xml")
NewsFeed = feedparser.parse("./archive_feed")

N = len(NewsFeed.entries)
titles = [e.title for e in NewsFeed.entries]


In [192]:

currentadv = [n for n in range(N) if 'Public Advisory' in titles[n]]
if len(currentadv)==0:
    current = ''
    
    currentinfo = pd.DataFrame([{
            'datetime': '',
            'current_latitude': '',
            'current_longitude': '',
            'winds_mph': '',
            'storm_category': 'NO CURRENT STORM',
            'pressure_mbar': '',
            'movement_direction': '',
            'movement_speed': '',
            'watch_updates': '',
            'warning_updates': '',
            'alerts_summary': ''
        }])

else:
    entries = []
    for n in range(len(currentadv)):
        currentadv = currentadv[n]
        current = ''.join(NewsFeed.entries[currentadv].summary.split('\n'))
        current = current.replace('--','')
        current = ''.join([c.upper() for c in current])

        pressure = current.split('PRESSURE...')[1].split('...')[0]
        winds = current.split('WINDS...')[1].split('...')[0]
        windval = int(winds.split(' ')[0])

        if windval>=157:   category = 'Cat 5 Hurricane' 
        elif windval>=130: category = 'Cat 4 Hurricane'
        elif windval>=111: category = 'Cat 3 Hurricane'
        elif windval>=96:  category = 'Cat 2 Hurricane'
        elif windval>=74:  category = 'Cat 1 Hurricane'
        elif windval>=39:  category = 'Tropical Storm'
        else:              category = 'Subtropical'

        location = current.split('LOCATION...')[1].split(' ABOUT')[0].split(' ')
        latitude = location[0]
        longitude = location[1]

        issuetime = current.split('ISSUED AT ')[1].split(' <PRE')[0]

        movement = current.split('MOVEMENT...')[1].split('...')[0].split(' OR ')[1].split(' AT ')
        pathdirection = movement[0]
        pathspeed = movement[1]

        warningtext = current.split('CHANGES WITH THIS ADVISORY... ')[1].split(' DISCUSSION')[0]
        warningtext = warningtext.replace('U.S.','US')
        warningtext = warningtext.replace('...','-')
        warningtext = warningtext.replace(' A TROPICAL STORM WARNING MEANS THAT TROPICAL STORM CONDITIONS ARE EXPECTED SOMEWHERE WITHIN THE WARNING AREA WITHIN 36 HOURS.','')
        warningtext = warningtext.replace(' A TROPICAL STORM WATCH MEANS THAT TROPICAL STORM CONDITIONS ARE POSSIBLE WITHIN THE WATCH AREA-GENERALLY WITHIN 48 HOURS.','')
        warningtext = warningtext.replace(' FOR STORM INFORMATION SPECIFIC TO YOUR AREA-INCLUDING POSSIBLE INLAND WATCHES AND WARNINGS-PLEASE MONITOR PRODUCTS ISSUED BY YOUR LOCAL NATIONAL WEATHER SERVICE FORECAST OFFICE.','')

        alerts = warningtext.split('.')
        summarytext   = ''.join([c for c in alerts if ('SUMMARY' in c)])
        watchtext   = ''.join([c for c in alerts if ('WATCH' in c) and ('SUMMARY' not in c)])
        warningtext = ''.join([c for c in alerts if ('WARNING' in c) and ('SUMMARY' not in c)])
        if (watchtext=='') and (warningtext==''): summarytext = 'NO CHANGES: ' + summarytext

        entries.append({
            'datetime': issuetime,
            'current_latitude': latitude,
            'current_longitude': longitude,
            'winds_mph': winds,
            'storm_category': category,
            'pressure_mbar': pressure,
            'movement_direction': pathdirection,
            'movement_speed': pathspeed,
            'watch_updates': watchtext,
            'warning_updates': warningtext,
            'alerts_summary': summarytext
        })
    currentinfo = pd.DataFrame(entries)
    
    
currentinfo

,datetime,current_latitude,current_longitude,winds_mph,storm_category,pressure_mbar,movement_direction,movement_speed,watch_updates,warning_updates,alerts_summary
0,600 PM EDT WED JUN 05 2013,25.3N,86.5W,40 MPH,Tropical Storm,1002 MB,360 DEGREES,3 MPH,A TROPICAL STORM WATCH HAS BEEN ISSUED FOR TH...,A TROPICAL STORM WARNING HAS BEEN ISSUED FOR T...,SUMMARY OF WATCHES AND WARNINGS IN EFFECT- A ...


In [193]:

currentinfo.to_csv('./currentstorminfo.csv', index=False)

For future development:

In [175]:
import requests
base_url = 'https://api.weather.gov/'
response = requests.get(base_url + 'alerts/active?urgency=expected')
# for everything:
response.json()
#real alerts:
L = len(response.json()['features'])
hurricanealerts = [n for n in range(L) if 'Hurricane' in response.json()['features'][n]['properties']['event']]
tropstormalerts = [n for n in range(L) if 'Tropical' in response.json()['features'][n]['properties']['event']]
windalerts = [n for n in range(L) if 'Wind' in response.json()['features'][n]['properties']['event']]
windalerts

[147]

In [176]:
chk = response.json()['features'][147]
chk['properties']['event']

'Lake Wind Advisory'

In [169]:
# neat! all the types of alerts:
response = requests.get(base_url + 'alerts/types')
response.json()['eventTypes']

['911 Telephone Outage Emergency',
 'Administrative Message',
 'Air Quality Alert',
 'Air Stagnation Advisory',
 'Arroyo And Small Stream Flood Advisory',
 'Ashfall Advisory',
 'Ashfall Warning',
 'Avalanche Advisory',
 'Avalanche Warning',
 'Avalanche Watch',
 'Beach Hazards Statement',
 'Blizzard Warning',
 'Blizzard Watch',
 'Blowing Dust Advisory',
 'Blowing Dust Warning',
 'Brisk Wind Advisory',
 'Child Abduction Emergency',
 'Civil Danger Warning',
 'Civil Emergency Message',
 'Coastal Flood Advisory',
 'Coastal Flood Statement',
 'Coastal Flood Warning',
 'Coastal Flood Watch',
 'Dense Fog Advisory',
 'Dense Smoke Advisory',
 'Dust Advisory',
 'Dust Storm Warning',
 'Earthquake Warning',
 'Evacuation - Immediate',
 'Excessive Heat Warning',
 'Excessive Heat Watch',
 'Extreme Cold Warning',
 'Extreme Cold Watch',
 'Extreme Fire Danger',
 'Extreme Wind Warning',
 'Fire Warning',
 'Fire Weather Watch',
 'Flash Flood Statement',
 'Flash Flood Warning',
 'Flash Flood Watch',
 'Flood 